In [5]:
import os
import openai

from langchain.chat_models import AzureChatOpenAI

# Setting the env variables

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# variables
openai.api_key = os.getenv("api_key")
openai.api_base =  os.getenv("api_base") 
openai.api_type = os.getenv("api_type")
openai.api_version = os.getenv("api_version") # this may change in the future
deployment_id=os.getenv("deployment_id_gpt_4") 

gpt_model = os.getenv("deployment_id_gpt_4") 
model_engine = os.getenv("deployment_id_gpt_4") 
embd_model = 'text-embedding-ada-002'

os.environ["OPENAI_API_TYPE"] = os.getenv("api_type")
os.environ["OPENAI_API_VERSION"] = os.getenv("api_version")
os.environ["OPENAI_API_BASE"] = os.getenv("api_base")
os.environ["OPENAI_API_KEY"] = os.getenv("api_key")

SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

In [8]:
llm = AzureChatOpenAI(deployment_name=deployment_id,model_name=deployment_id, temperature=0.2)

# initialize a calculator tool using a LLMMathChain

In [10]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

In [12]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [15]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
	agent="zero-shot-react-description",
	tools=tools,
	llm=llm,
	verbose=True,
	max_iterations=8
)

In [14]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I need to calculate the expression (4.5*2.1)^2.2. First, I will calculate the product of 4.5 and 2.1.
Action: Calculator
Action Input: 4.5 * 2.1
Observation: Answer: 9.450000000000001
Thought:Now that I have the product of 4.5 and 2.1, I need to raise it to the power of 2.2.
Action: Calculator
Action Input: 9.450000000000001 ** 2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer.
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [16]:
(4.5*2.1)**2.2

139.94261298333066

# Another Example... bit complex

In [18]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple " \
                "boxes (apple box contains eight apples), how many apples do we " \
                "have?")



> Entering new AgentExecutor chain...
I need to calculate the total number of apples by adding Mary's apples and the apples from Giorgio's apple boxes.
Thought: First, I need to find out how many apples are in two and a half apple boxes.
Action: Calculator
Action Input: 2.5 * 8
Observation: Answer: 20.0
Thought:Now I know there are 20 apples in Giorgio's apple boxes. I need to add this to Mary's apples to find the total number of apples.
Action: Calculator
Action Input: 4 + 20
Observation: Answer: 24
Thought:I now know the final answer.
Final Answer: 24

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': '24'}

## But what if we ask a non-math question? Something that a plain and simple LLM should be able to answer easily?

In [19]:
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
I cannot answer this question using a calculator, but I know the answer from my knowledge base.
Final Answer: Oslo

> Finished chain.


{'input': 'what is the capital of Norway?', 'output': 'Oslo'}

# the agent to answer this "general knowledge" question. We can, we just need to link it to an LLM tool.

In [20]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

# Add the new tool to our tools list:

In [21]:
tools.append(llm_tool)

# Now we reinitialize the agent with our two tools ("Calculator" and "Language Model")

In [26]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=8
)

In [23]:
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
I need to use the Language Model to find the capital of Norway.
Action: Language Model
Action Input: What is the capital of Norway?
Observation: The capital of Norway is Oslo.
Thought:I now know the final answer.
Final Answer: Oslo

> Finished chain.


{'input': 'what is the capital of Norway?', 'output': 'Oslo'}

In [24]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I need to calculate the expression (4.5*2.1)^2.2. I will use the calculator to compute the result.
Action: Calculator
Action Input: (4.5*2.1)**2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer.
Final Answer: 139.94

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94'}

In [28]:
zero_shot_agent("what is the square root of the multiplication of populations of India and Canada?")



> Entering new AgentExecutor chain...
First, I need to find the populations of India and Canada. Then, I will multiply them and find the square root of the result. 
Action: Language Model
Action Input: What are the populations of India and Canada?
Observation: As of October 2021, the population of India is approximately 1.366 billion, while the population of Canada is approximately 38 million. Please note that these figures are estimates and may change over time.
Thought:Now I have the population numbers, I will multiply them and find the square root of the result.
Action: Calculator
Action Input: 1366000000 * 38000000
Observation: Answer: 51908000000000000
Thought:Now I have the product of the populations, I will find the square root of this number.
Action: Calculator
Action Input: sqrt(51908000000000000)
Observation: Answer: 227833272.3725839
Thought:I now know the final answer.
Final Answer: 227,833,272.37

> Finished chain.


{'input': 'what is the square root of the multiplication of populations of India and Canada?',
 'output': '227,833,272.37'}